In [206]:
from selenium import webdriver
import urllib.request
import time
import random

from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys

from elasticsearch import Elasticsearch

In [207]:
LOCAL = True
es_client = Elasticsearch(hosts=["localhost" if LOCAL else "elasticsearch"])

In [3]:
IS_LINUX = False

In [4]:
driver = webdriver.Firefox(executable_path="geckodriver.exe" if not IS_LINUX else "./chromedriver_linux")

##UA = 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:25.0) Gecko/20100101 Firefox/25.0'
#PHANTOMJS_ARG = {'phantomjs.page.settings.userAgent': UA}

#driver = webdriver.PhantomJS(desired_capabilities=PHANTOMJS_ARG)

In [5]:
driver.get("https://tinder.com/app/recs")

### Connexion 

In [6]:
classbutton = "button Lts($ls-s) Z(0) CenterAlign Mx(a) Cur(p) Tt(u) Ell Px(24px) Px(20px)--s Py(0) Bdrs(0) Mih(40px) Fw($semibold) focus-button-style Fz($s) Bdrs(4px) Bg(#fff) C($c-pink) Bg($primary-gradient):h C(#fff):h"
python_button = driver.find_element_by_xpath('//*[@class="'+ classbutton +'"]')
python_button.click()

In [7]:
connect_phone_number = "Connexion avec un numéro de tél."
python_button = driver.find_element_by_xpath('//*[@aria-label="'+ connect_phone_number +'"]')
python_button.click()

In [8]:
driver.switch_to.default_content()
driver.switch_to.frame(driver.find_element_by_tag_name("iframe"))
driver.switch_to.frame(driver.find_element_by_tag_name("iframe"))
driver.switch_to.frame(driver.find_element_by_tag_name("iframe"))

In [9]:
python_button = driver.find_element_by_id("home_children_button")
python_button.click()

### TakeCaptcha
Ignorer cette cell

In [11]:
i = 11

img = driver.find_element_by_id("game_challengeItem_image")
src = img.get_attribute('src')
name = "image_0" + str(i) + ".png"
urllib.request.urlretrieve(src, name)
i += 1

filedata = urllib.request.urlopen(src)
datatowrite = filedata.read()
 
with open('/projet/ESGI/3ESGI/NoSQL/BotCapcha/' + name, 'wb') as f:
    f.write(datatowrite)

In [10]:
driver.switch_to.default_content()
phone_element = driver.find_element_by_name("phone_number")
phone_element.send_keys("0695776028")

In [165]:
def TakeInstagram():
    try:
        class_desc = "P(16px) Ta(start) Us(t) C($c-secondary) BreakWord Whs(pl) Fz($ms)"
        takedesc = driver.find_element_by_xpath('//*[@class="'+ class_desc +'"]')
        takedesc = takedesc.find_elements_by_css_selector("*")

        insta_list = [ "📷 ", "📸 ","@",
            "IG: ", "Ig: ","ig: ", "insta: ", "instagram: ", "Insta: ", "Instagram: ", "INSTA: ", "INSTAGRAM: ",
            "IG, ", "Ig, ","ig, ", "insta, ", "instagram, ", "Insta, ", "Instagram, ", "INSTA, ", "INSTAGRAM: ",
            "IG,", "Ig,","ig,", "insta,", "instagram,", "Insta,", "Instagram,", "INSTA,", "INSTAGRAM,",
            "IG:", "Ig:","ig:", "insta:", "instagram:", "Insta:", "Instagram:", "INSTA:", "INSTAGRAM:",
            "IG : ", "Ig : ", "ig : ", "insta : ", "instagram : ", "Insta : ", "Instagram : ", "INSTA : ", "INSTAGRAM : ",
            "IG :", "Ig :","ig :", "insta :", "instagram :", "Insta :", "Instagram :", "INSTA :", "INSTAGRAM :",
            "IG ", "Ig ","ig ", "insta ", "instagram ", "Insta ", "Instagram ", "INSTA  ", "INSTAGRAM "]

        description = takedesc[0].text
        instagram = ""
        description = description.split("\n")

        for desc_split in description:

            for insta_occurent in insta_list:

                result = desc_split.find(insta_occurent, 0, len(desc_split))
                if result != -1:
                    start = result + len(insta_occurent)
                    instagram = desc_split[start: len(desc_split)]
                    break

        instagramlist = instagram.split(" ")

        for word in instagramlist:
            
            if word != " " and word != "" and word != "➡️":
                instagram = word;
                break
        instagram = instagram.replace("(", "").replace(")", "")
        return instagram
    
    except:
        
        return ""

In [12]:
def TakeTag():
    try:
        class_tag = "Mt(4px)"
        taketag = driver.find_element_by_xpath('//*[@class="'+ class_tag +'"]')
        taketag = taketag.find_elements_by_css_selector("*")
        
        list_tag = []
        
        for tag in taketag:
            list_tag.append(tag.text)
        
        
        return list_tag
    
    except:
        
        return []

In [13]:
def TakeName():
    try:
        class_name = "Fz($xl) Fw($bold) Fxs(1) Fxw(w) Pend(8px) M(0) D(i)"
        takename = driver.find_element_by_xpath('//*[@class="'+ class_name +'"]')
        
        return takename.text
    
    except:
        
        return ""

In [14]:
def TakeAge():
    try:
        class_age = "Whs(nw) Fz($l)"
        takeage = driver.find_element_by_xpath('//*[@class="'+ class_age +'"]')
        
        return takeage.text
    
    except:
        
        return ""

In [15]:
def TakePicture():
    try:
        class_picture = "profileCard__slider__img Z(-1)"
        takepicture = driver.find_element_by_xpath('//*[@class="'+ class_picture +'"]')
        style = takepicture.get_attribute("style")
        
        url = style[style.find("url") + 5: len(style)].split("\")")[0]
        
        return url
    
    except:
        
        return ""

In [16]:
def PushDataBase(data):
    es_client.index(index="tinderplusplus", doc_type='algo', id=None, body=data)

In [195]:
actions = ActionChains(driver)
actions.send_keys(Keys.ARROW_UP + 't')
actions.perform()
insta = TakeInstagram()
if insta != "":
    tag = TakeTag()
    name = TakeName()
    age = TakeAge()
    picture = TakePicture()
    
    data = {
        "name": name,
        "instagram": insta,
        "age": age,
        "picture": picture,
        "tag": tag
        
    }
    print(data)
    PushDataBase(data)
    
rnd = random.randrange(1,5)   

#time.sleep(rnd)
#actions.send_keys(Keys.ARROW_RIGHT + 't')
#actions.perform()   

{'name': 'Noa', 'instagram': 'itsnoacarla', 'age': '20', 'picture': 'https://images-ssl.gotinder.com/u/vohGc8UzJigNpRcDmJmLVi/bLFgroGSzJKNmQHFu9e3bs.jpg?Policy=eyJTdGF0ZW1lbnQiOiBbeyJSZXNvdXJjZSI6IiovdS92b2hHYzhVekppZ05wUmNEbUptTFZpLyoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE2MTg5MzAxOTh9fX1dfQ__&Signature=G3GG7fZwjWXN-k9RxsLubFg1qXUndYd9oXrfZf9YjNrqbwujVKNA2jffVKRNEbywkFn2t96KCoGf7IWH0aShK11kLPHlq28cCsDDgQiAcu3H7FZ~BJRJorPposUBVdvBMgihXf3IT483DWyfVadBxCEqSUT7NOyaiBfZsCLO5gGECKEjZPqCPzvAzDx4uRAGu17GL390lutWvF7nqpcLeDEYlzeMtxLRf25Uud1r8jzxgADVfRV0zPIyzl6wwoRVP0IzCNEdwlvz8HMdkFoqzSQdE7krBganLLrO~PsIrTTNZCAgB7K5~5Bc47OX85D2JFr8j1AxDMzVClDU~biXCw__&Key-Pair-Id=K368TLDEUPA6OI', 'tag': ['Art', 'Tatouages', 'Mode']}


### View Result

In [209]:
es_client.count(index='tinderplusplus', doc_type='algo')["count"]

50

In [208]:
try:
    result = es_client.search(index="tinderplusplus", body={"query": {"match_all": {}}})
    ids = []
    for hit in result['hits']['hits']:
        print("Name : {name}\nAge : {age}\nPicture : {picture} \nInstagram : {instagram} \nTags : {tag} \n".format(**hit['_source']))
        print("******************")
        ids.append(hit["_id"])
except:
     None

Name : Oriane
Age : 19
Picture : https://images-ssl.gotinder.com/u/bbSsuciPyd5bHCYsE4hvHM/gH811aUUhfpa8sL9FEbXVJ.jpg?Policy=eyJTdGF0ZW1lbnQiOiBbeyJSZXNvdXJjZSI6IiovdS9iYlNzdWNpUHlkNWJIQ1lzRTRodkhNLyoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9jaFRpbWUiOjE2MTg5MjcwMjR9fX1dfQ__&Signature=HKTg46pwYSORL-2PsraiC-6rGC-UxwYev0uXKWix3iLcBmwJNrQ7Q17nxCBxn7IzYxrKEoeRzC-s6F-CkhPIxPbSbwEtG9cJ5CbmEdPEbFlSUP5UNkFrzA4jbxkVk6gxC4r3U5SH7JcJo8aDtqVQQwDLG8puEzmHyPSFxfFmQprwtoGv0Gp4yHES5mdZ5o6yJZ2fLqFid2kb~zet~TKYakLtnhC86yybdWe8lZNbVNoPWU7gQVqa92~yT1wKniR2baYKlehvy7eklV2vKgrpr~iWSpwAaVzgpYoA8EHvYkF0pMfj5v4lC~p2n5idqM5ow7A1x1V1dHt6albCEvuyIQ__&Key-Pair-Id=K368TLDEUPA6OI 
Instagram : oriane_sp 
Tags : ['Tatouages', 'Prendre un verre', 'Rencontres sans pression'] 

******************
Name : Mathilde
Age : 19
Picture : https://images-ssl.gotinder.com/u/t39V8FaA1KAo24toDFXLWW/721tUc8GtwstEFjw6TGaes.jpg?Policy=eyJTdGF0ZW1lbnQiOiBbeyJSZXNvdXJjZSI6IiovdS90MzlWOEZhQTFLQW8yNHRvREZYTFdXLyoiLCJDb25kaXRpb24iO

### DELETE ALL 

In [191]:
#es_client.indices.delete(index='tinderplusplus')

### CLOSE BROWSER

In [140]:
driver.close()